In [1]:
import os, shutil

In [2]:
os.listdir('noticias')

['ciencias4.txt',
 'deportes3.txt',
 'deportes6.txt',
 'ciencias6.txt',
 'economia4.txt',
 'ciencias3.txt',
 'ciencias2.txt',
 'deportes2.txt',
 'economia5.txt',
 'economia2.txt',
 'deportes1.txt',
 'deportes5.txt',
 'economia3.txt',
 'economia1.txt',
 'ciencias1.txt',
 'deportes4.txt',
 'ciencias5.txt',
 'economia6.txt']

In [3]:
docs_list = []
for i, noti in enumerate(os.listdir('noticias')):
    docs_list.append('')
    with open("noticias/"+noti) as f:
        for line in f:
            docs_list[i] = docs_list[i] + line.replace('\n',' ')
    f.close()
len(docs_list)

18

In [4]:
stop_list = []
with open("stopwords.txt") as f:
    for line in f:
        stop_list.append(line.replace('\n', ''))

### https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24
https://towardsdatascience.com/topic-modeling-for-the-new-york-times-news-dataset-1f643e15caac

In [5]:
doc_complete = docs_list
import gensim
from gensim import corpora, models
# from gensim.utils import simple_preprocess
# from gensim.parsing.preprocessing import STOPWORDS
# from gensim import parsing

import nltk
from nltk.stem.snowball import SpanishStemmer
from nltk import PorterStemmer
from nltk.stem.porter import *
#need to install this library with this comand ->
#nltk.download()
#nltk.download('stopwords')
from nltk.corpus import stopwords 
#nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
#nltk.download('punkt')
from nltk.tokenize import word_tokenize
import numpy as np
import string

#stem might not be an actual word
stemmer = SpanishStemmer()

#lemma is an actual language word
lemma = WordNetLemmatizer()

#stop word is a commonly used word (such as “the”, “in”) that a search engine has been programmed to ignore
stop = set(stopwords.words('spanish'))
stop = list(set().union(stop,stop_list))
#puntuación a eliminar
exclude = set().union(string.punctuation,'“”¿|/\ºª@#~$€%½¬&{}[]()=`+*-¨´Çç_-:.;,¡!')

#function to stem
# def stemmer_stemming(text):
#     return stemmer.stem(text)

def clean(doc):
#     print(doc)
    #Quitamos las stop words y aplicamos metodo lower
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop and len(i) > 3])
    #removemos la puntuancion
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in word_tokenize(punc_free))
#     print(normalized)
#     result = []
#     for token in gensim.utils.simple_preprocess(normalized):
#         if token not in gensim.parsing.preprocessing.STOPWORDS:
#             result.append(stemmer_stemming(token))
#     print(result)
    return normalized

#limpiamos la lista de documentos
doc_clean = [clean(doc).split() for doc in doc_complete]

#Creamos un diccionario a partir de 'doc_clean' que contenga el número de veces que una palabra aparece
dictionary = corpora.Dictionary(doc_clean)

#Filtramos los tokens que aparecen en:
#     menos de n documentos (número absoluto) o
#     más de % documentos (fracción del tamaño total del corpus, no número absoluto).
#     después de los dos pasos anteriores, guarde sólo los primeros 5000 tokens más frecuentes.
dictionary.filter_extremes(no_below=3, no_above=0.7, keep_n=1000)

# Para cada documento creamos un diccionario de palabras con las veces que aparecen
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

#################
#check dictionary
# print('Palabras que más aparecen: ')
# counter_dic = 1
# for i in doc_term_matrix:
#     for k in i:
#         if k[1] > 3:
#             print("Palabra {} (\"{}\") aparece {} veces.".format(k[0],
#                                                          dictionary[k[0]],
#                                                          k[1]))
#         counter_dic +=1
# # counter_dic
#################
Nro_topis = 8
Nro_words = 4

#algoritmo LDA usando un diccionario de palabras
print("Algoritmo LDA usando un diccionario de palabras: ")
Lda = gensim.models.LdaMulticore
lda_model = Lda(doc_term_matrix, num_topics=Nro_topis, id2word = dictionary, passes=30, alpha=.01, eta=.01, workers=3)

for idx, topic in lda_model.print_topics(num_topics=Nro_topis, num_words=Nro_words):
    print('Topic: {} \nWords: {}'.format(idx+1, topic))

# lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)
print()
print("")
#creamos un objeto tf-idf "Term frequency – Inverse document frequency"
tfidf = models.TfidfModel(doc_term_matrix)
corpus_tfidf = tfidf[doc_term_matrix]

#algoritmo LDA usando un objeto tf-idf "Term frequency – Inverse document frequency"
print("Algoritmo LDA usando un objeto tf-idf: ")
lda_model_tfidf = Lda(corpus_tfidf, num_topics=Nro_topis, id2word = dictionary, passes=30, alpha=.0005, eta=.005, workers=3)

for idx, topic in lda_model_tfidf.print_topics(num_topics=Nro_topis, num_words=Nro_words):
    print('Topic: {} \nWord: {}'.format(idx+1, topic))

Algoritmo LDA usando un diccionario de palabras: 


/home/joseg/miniconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


Topic: 1 
Words: 0.070*"compañía" + 0.057*"millones" + 0.045*"mercado" + 0.040*"firma"
Topic: 2 
Words: 0.045*"fútbol" + 0.043*"técnico" + 0.035*"hora" + 0.033*"selección"
Topic: 3 
Words: 0.004*"hombre" + 0.004*"economía" + 0.004*"negocio" + 0.004*"rivales"
Topic: 4 
Words: 0.040*"agujeros" + 0.038*"científicos" + 0.036*"einstein" + 0.036*"millones"
Topic: 5 
Words: 0.046*"américa" + 0.034*"fútbol" + 0.033*"ingresos" + 0.030*"países"
Topic: 6 
Words: 0.083*"internacional" + 0.057*"científicos" + 0.051*"base" + 0.051*"me"
Topic: 7 
Words: 0.053*"negocio" + 0.040*"economía" + 0.040*"hombre" + 0.026*"mercado"
Topic: 8 
Words: 0.004*"gravedad" + 0.004*"millones" + 0.004*"científicos" + 0.004*"einstein"


Algoritmo LDA usando un objeto tf-idf: 
Topic: 1 
Word: 0.158*"internacional" + 0.064*"radiación" + 0.061*"medida" + 0.054*"vivir"
Topic: 2 
Word: 0.053*"fútbol" + 0.046*"argentina" + 0.040*"técnico" + 0.039*"selección"
Topic: 3 
Word: 0.029*"ingresos" + 0.024*"agujeros" + 0.023*"científi

# Probando en test 

## Economia

In [6]:
docs_list_test = ['']
with open("test_eco.txt") as f:
    for line in f:
        docs_list_test[0] = docs_list_test[0] + line.replace('\n',' ')
    f.close()

In [7]:
doc_clean_test = [clean(doc).split() for doc in docs_list_test]
doc_term_matrix_test = [dictionary.doc2bow(doc) for doc in doc_clean_test]

#### lda_model

In [8]:
bow_vector = doc_term_matrix_test[0]
for index, score in sorted(lda_model[bow_vector], key=lambda x: -1*x[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, Nro_words)))

Score: 0.5114386677742004	 Topic: 0.070*"compañía" + 0.057*"millones" + 0.045*"mercado" + 0.040*"firma"
Score: 0.33321481943130493	 Topic: 0.046*"américa" + 0.034*"fútbol" + 0.033*"ingresos" + 0.030*"países"
Score: 0.09188761562108994	 Topic: 0.083*"internacional" + 0.057*"científicos" + 0.051*"base" + 0.051*"me"
Score: 0.06266015022993088	 Topic: 0.040*"agujeros" + 0.038*"científicos" + 0.036*"einstein" + 0.036*"millones"


#### lda_model_tfidf

In [9]:
for index, score in sorted(lda_model_tfidf[bow_vector], key=lambda x: -1*x[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, Nro_words)))


Score: 0.3965222239494324	 
Topic: 0.029*"ingresos" + 0.024*"agujeros" + 0.023*"científicos" + 0.022*"millones"

Score: 0.28827670216560364	 
Topic: 0.044*"américa" + 0.044*"persona" + 0.041*"mercado" + 0.040*"universidad"

Score: 0.12465716153383255	 
Topic: 0.064*"firma" + 0.064*"empleados" + 0.049*"rivales" + 0.043*"éxito"

Score: 0.10639659315347672	 
Topic: 0.053*"fútbol" + 0.046*"argentina" + 0.040*"técnico" + 0.039*"selección"

Score: 0.08411730825901031	 
Topic: 0.096*"hombre" + 0.060*"latina" + 0.055*"presidente" + 0.042*"mujer"


## Deporte

In [10]:
docs_list_test = ['']
with open("test_dep.txt") as f:
    for line in f:
        docs_list_test[0] = docs_list_test[0] + line.replace('\n',' ')
    f.close()

In [11]:
doc_clean_test = [clean(doc).split() for doc in docs_list_test]
doc_term_matrix_test = [dictionary.doc2bow(doc) for doc in doc_clean_test]

#### lda_model

In [12]:
bow_vector = doc_term_matrix_test[0]
for index, score in sorted(lda_model[bow_vector], key=lambda x: -1*x[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, Nro_words)))

Score: 0.5249103903770447	 Topic: 0.045*"fútbol" + 0.043*"técnico" + 0.035*"hora" + 0.033*"selección"
Score: 0.3563339114189148	 Topic: 0.046*"américa" + 0.034*"fútbol" + 0.033*"ingresos" + 0.030*"países"
Score: 0.11719703674316406	 Topic: 0.040*"agujeros" + 0.038*"científicos" + 0.036*"einstein" + 0.036*"millones"


#### lda_model_tfidf

In [13]:
for index, score in sorted(lda_model_tfidf[bow_vector], key=lambda x: -1*x[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, Nro_words)))


Score: 0.43705153465270996	 
Topic: 0.053*"fútbol" + 0.046*"argentina" + 0.040*"técnico" + 0.039*"selección"

Score: 0.38421374559402466	 
Topic: 0.029*"ingresos" + 0.024*"agujeros" + 0.023*"científicos" + 0.022*"millones"

Score: 0.08766701817512512	 
Topic: 0.096*"hombre" + 0.060*"latina" + 0.055*"presidente" + 0.042*"mujer"

Score: 0.05971253290772438	 
Topic: 0.064*"firma" + 0.064*"empleados" + 0.049*"rivales" + 0.043*"éxito"

Score: 0.03130831941962242	 
Topic: 0.158*"internacional" + 0.064*"radiación" + 0.061*"medida" + 0.054*"vivir"


## Ciencia

In [14]:
docs_list_test = ['']
with open("test_cie.txt") as f:
    for line in f:
        docs_list_test[0] = docs_list_test[0] + line.replace('\n',' ')
    f.close()

In [15]:
doc_clean_test = [clean(doc).split() for doc in docs_list_test]
doc_term_matrix_test = [dictionary.doc2bow(doc) for doc in doc_clean_test]

#### lda_model

In [16]:
bow_vector = doc_term_matrix_test[0]
for index, score in sorted(lda_model[bow_vector], key=lambda x: -1*x[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, Nro_words)))

Score: 0.7169049382209778	 Topic: 0.040*"agujeros" + 0.038*"científicos" + 0.036*"einstein" + 0.036*"millones"
Score: 0.19706152379512787	 Topic: 0.046*"américa" + 0.034*"fútbol" + 0.033*"ingresos" + 0.030*"países"
Score: 0.0856606513261795	 Topic: 0.053*"negocio" + 0.040*"economía" + 0.040*"hombre" + 0.026*"mercado"


#### lda_model_tfidf

In [17]:
for index, score in sorted(lda_model_tfidf[bow_vector], key=lambda x: -1*x[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, Nro_words)))


Score: 0.6611242890357971	 
Topic: 0.029*"ingresos" + 0.024*"agujeros" + 0.023*"científicos" + 0.022*"millones"

Score: 0.10835292190313339	 
Topic: 0.044*"américa" + 0.044*"persona" + 0.041*"mercado" + 0.040*"universidad"

Score: 0.08942861109972	 
Topic: 0.160*"gravedad" + 0.081*"mile" + 0.076*"realmente" + 0.071*"obtener"

Score: 0.07408390194177628	 
Topic: 0.053*"fútbol" + 0.046*"argentina" + 0.040*"técnico" + 0.039*"selección"

Score: 0.04173697158694267	 
Topic: 0.096*"hombre" + 0.060*"latina" + 0.055*"presidente" + 0.042*"mujer"

Score: 0.019295262172818184	 
Topic: 0.158*"internacional" + 0.064*"radiación" + 0.061*"medida" + 0.054*"vivir"


## Fuentes

Train:
- deportes1 https://www.bbc.com/mundo/deportes-47874344
- deportes2 https://www.bbc.com/mundo/deportes-47867626
- deportes3 https://www.bbc.com/mundo/deportes-47867623
- deportes4 https://www.bbc.com/mundo/deportes-47867621
- deportes5 https://www.bbc.com/mundo/deportes-47827846
- deportes6 https://www.bbc.com/mundo/noticias-47758928
- ciencias1 https://www.bbc.com/mundo/noticias-47915625
- ciencias2 https://www.bbc.com/mundo/noticias-47859720
- ciencias3 https://www.bbc.com/mundo/noticias-47915620
- ciencias4 https://www.bbc.com/mundo/noticias-47909000
- ciencias5 https://www.bbc.com/mundo/noticias-47860116
- ciencias6 https://www.bbc.com/mundo/noticias-47900442
- economia1 https://www.bbc.com/mundo/noticias-47861934
- economia2 https://www.bbc.com/mundo/noticias-47670796
- economia3 https://www.bbc.com/mundo/noticias-47901139
- economia4 https://www.bbc.com/mundo/noticias-47908922
- economia5 https://www.bbc.com/mundo/noticias-47900832
- economia6 https://www.bbc.com/mundo/noticias-47895086


Test:
- test_cie https://www.bbc.com/mundo/noticias-47884630
- test dep https://www.bbc.com/mundo/deportes-47667089
- test_eco https://www.bbc.com/mundo/noticias-47872006